# Transmisión de señales por un canal de comunicaciones
<p><code>Python en Jupyter Notebook</code></p>
<p>Creado por <code>Giancarlo Ortiz</code> para explicar los fundamentos de los <code>Sistemas de comunicaciones</code> en los cursos de telemática y redes de computadores.</p>

## Descripción
Este algoritmo, también llamado de caminos mínimos, es un método para resolver el problema del camino mas corto.

### Agenda
1. Factores que afectan la transmisión.
1. Tasas de transferencia.
1. Capacidad del canal.

In [1]:
# Importar módulos al cuaderno
import math as m
import matplotlib.pyplot as plt

# Importar elementos de un módulo al cuaderno
from math import pi
from numpy import sin

## 1. Factores que afectan la transmisión  ✔

### 1.1. La atenuación 
la [Atenuación][alpha] es perdida de potencia de la señal durante su propagación por el medio, depende de las características del medio y de la frecuencia.


[alpha]:https://es.wikipedia.org/wiki/Atenuación

#### 1.1.1. Causas:
* Absorción del material.
* Oposición Transmisión.
* Defectos de fabricación.
* Deformación del material.

#### 1.1.2. Representación:

![Representación](../img/attenuation_utp.svg "Atenuación")

#### 1.1.3. Ecuaciones:
$$
\begin{aligned}
\alpha  &= 10 \cdot log_{10}\left(\frac{P_o}{P_i}\right) \\ 
        &= 20 \cdot log_{10}\left(\frac{V_o}{V_i}\right) \\
\end{aligned}
$$

In [2]:
def voltage_attenuation(voltage_out:float, voltage_in:float):
    ''' Calcula la atenuación en un medio de txx desde los valores de voltaje de la señal de entrada. '''
    return round(20 * m.log10(voltage_out/voltage_in))

def voltage_out(decibels:float, voltage_in:float=100):
    ''' Calcula el voltaje de salida desde los valores de atenuación de la señal de entrada. '''
    return round(voltage_in * (10 ** (decibels/20)), 2)

def power_attenuation(power_out:float, power_in:float):
    ''' Calcula la atenuación desde los valores de potencia de la señal de entrada. '''
    return round(10 * m.log10(power_out/power_in))

def power_out(decibels:float, power_in:float=100):
    ''' Calcula la potencia de salida desde los valores de atenuación de la señal de entrada. '''
    return round(power_in * (10 ** (decibels/10)), 2)

# Salida estándar - Formateo de cadenas
print(20*"--")
print(f"Atenuación de voltaje:  {voltage_attenuation(70.7, 100)} dB")
print(f"Voltaje de salida: {voltage_out(-30)} voltios")
print(20*"--")
print(f"Atenuación de potencia: {power_attenuation(50, 100)} dB")
print(f"Voltaje de salida: {power_out(-30)} vatios")
print(20*"--")

----------------------------------------
Atenuación de voltaje:  -3 dB
Voltaje de salida: 3.16 voltios
----------------------------------------
Atenuación de potencia: -3 dB
Voltaje de salida: 0.1 vatios
----------------------------------------


### 1.2. El ruido 
El [Ruido][noise] son las potencias parásitas que afectan la transmisión.

[noise]:https://es.wikipedia.org/wiki/Ruido_(comunicación)#

#### 1.2.1. relación señal a ruido
La [SNR][121] o relación señal/ruido o  se define como la proporción existente entre la potencia de la señal que se transmite y la potencia del ruido que la corrompe.

[121]:https://en.wikipedia.org/wiki/Signal-to-noise_ratio

$$
\begin{aligned}
SNR = \frac{P_s}{P_n}  &= \left(\frac{V_s}{V_n}\right)^{2} \\ 
     &= 10 \cdot log_{10}\left(\frac{P_s}{P_n}\right) = 20 \cdot log_{10}\left(\frac{V_s}{V_n}\right) \\
\end{aligned}
$$


In [3]:
def snr(señal:float, ruido:float, potencia:bool=True, dB:bool=True)->float:
    ''' Calcula la atenuación desde los valores de voltaje o potencia de la señal de entrada. '''
    ratio = señal/ruido
    lineal = ratio if potencia else ratio**2
    snr = round(10 * m.log10(lineal)) if dB else lineal
    return snr

voltaje_señal = 5
voltaje_ruido = 0.28
resistencia = 50 
potencia_señal = (voltaje_señal**2)/resistencia
potencia_ruido = (voltaje_ruido**2)/resistencia

# Salida estándar - Formateo de cadenas
print(26*"-")
print(f"Relación señal a ruido")
print(26*"-")
print(f"SNR voltaje:   {snr(voltaje_señal, voltaje_ruido, potencia=False, dB=False):11,.1f}")
print(f"SNR potencia:  {snr(potencia_señal, potencia_ruido, dB=False):11,.1f}")
print(f"SNR en dB:     {snr(potencia_señal, potencia_ruido):11,.2f} dB")
print(26*"-")

--------------------------
Relación señal a ruido
--------------------------
SNR voltaje:         318.9
SNR potencia:        318.9
SNR en dB:           25.00 dB
--------------------------


### 1.3. El eco
El [Eco][21_3] es reflexión de una parte de la potencia original.

[21_3]:https://es.wikipedia.org/wiki/P%C3%A9rdida_de_retorno

#### 1.3.1. Paradiafonia
La diafonía [NEXT][131] o de extremo cercano, denominada en inglés Near end crosstalk es la perturbación de un circuito a la señal de un circuito adyacente, medida en el extremo mas cercano a la fuente y la PSNEXT es una medida NEXT que incluye la suma algebraica de todas las contribuciones de diafonía de los pares adyacentes.

[131]:https://es.wikipedia.org/wiki/Diafon%C3%ADa#Paradiafon%C3%ADa

![Representación](../img/next.svg "Paradiafonia")

$$ \alpha_p = 20 \cdot log_{10}\left(\frac{V_{E}}{V_{I}}\right) $$


#### 1.3.2. Telediafonía
La diafonía [FEXT][132] o de extremo lejano, denominada en inglés Far end crosstalk es la perturbación de un circuito a la señal de un circuito adyacente, medida en el extremo mas lejano a la fuente y la PSFEXT es una medida FEXT que incluye la suma algebraica de todas las contribuciones de diafonía de los pares adyacentes

[132]:https://es.wikipedia.org/wiki/Diafon%C3%ADa#Telediafon%C3%ADa

![Representación](../img/fext.svg "Telediafonia")

$$ \alpha_t = 20 \cdot log_{10}\left(\frac{V_{E}}{V_{I}}\right) $$

>Nota: La AXT o Alien crosstalk es una interferencia provocada por otros cables colocados cerca del cable de interés, en contraposición a las señales contenidas en el mismo cable.


## 2. Tasas de transferencia  ✔
Es posible medir el rendimiento un canal de comunicaciones utilizando distintas definiciones; esto es porque la carga útil de mensajes (información) que la red es capaz de transportar por unidad de tiempo depende de la sobrecarga de los distintos protocolos en el modelo de comunicaciones, de las características del medio de transmisión y porque en algunos casos es necesario construir diferentes niveles de redundancia para garantizar una transmisión segura y libre de errores, lo que implica aumentar la sobrecarga de uno o varios protocolos encargados de la codificación y adaptación de la señal al medio.

### 2.1. Tasa de transferencia efectiva
La [tasa de transferencia efectiva][21_1] o throughput es el volumen de trabajo o de información neto que fluye a través de un sistema o red de computadoras; en esta medida se incluye toda la sobrecarga necesaria en los protocolos necesarios para una transmisión segura y libre de errores.

#### 2.1.1. Medidas relacionadas
* [Goodput][21_2]: es el rendimiento a nivel de aplicación de una comunicación, excluyendo la sobrecarga del protocolo.
* [Capacidad del canal][21_3]: es la medida de los recursos de comunicación teóricos aprovechables de una red telemática.
* [Ancho de Banda][21_4]: determina el rango de frecuencias aprovechables para la comunicación de un canal de comunicaciones.

#### 2.1.2. Comparación

__Goodput__ ([Carga util en bps][21_2]) <code>≤</code> __Throughput__ ([Velocidad real en bps][21_1]) <code>≤</code> __Channel capacity__ ([Capacidad teórica en bps][21_3])

![Representación](../img/channel.svg "Canal de comunicaciones")

[21_1]:https://es.wikipedia.org/wiki/Tasa_de_transferencia_efectiva
[21_2]:https://en.wikipedia.org/wiki/Goodput
[21_3]:https://es.wikipedia.org/wiki/Capacidad_del_canal
[21_4]:https://es.wikipedia.org/wiki/Ancho_de_banda

#### 2.1.3. Ejemplo: <code>HTTP</code> 
Para probar un canal de comunicaciones de 50 MB/s se necesitan un archivo de prueba de al menos 100 MB, que saturara el canal por al menos un segundo:

__Consideraciones:__
* HTTP 1.0:  Durante la carga de una página web de internet en el navegador, la descarga de cada elemento desde el servidor necesita la apertura y la clausura de una conexión TCP.
* HTTP 1.1:  El cliente y servidor pueden intercambiar mensajes sobre la misma conexión TCP.
* FTP:  La velocidad neta disponible para el usuario en una aplicación FTP, suele ser entre un 10% y un 15% menor a la velocidad real, debido al ancho de banda consumido por las cabeceras y las colas de los distintos protocolos usados en una transmisión libre de errores.
* TCP:  El tamaño de un paquete TCP esta limitado a 1500 bytes

In [14]:
# tcp/ip
bytes_tcp_max = 1500
encabezado_tcp = 20
encabezado_ip = 20
marca_de_tiempo = 12
carga_util_tcp = bytes_tcp_max - (encabezado_tcp + encabezado_ip + marca_de_tiempo)
eficiencia_tcp = 100*carga_util_tcp/bytes_tcp_max
# Ethernet
carga_util_eth = bytes_tcp_max
encabezado_trama = 26
brecha_entre_tramas = 96
bytes_eth_max = (encabezado_trama + brecha_entre_tramas) + bytes_tcp_max
eficiencia_eth = 100*carga_util_eth/bytes_eth_max
# Eficiencia total
eficiencia = eficiencia_eth*eficiencia_tcp/100

# Salida estándar - Formateo de cadenas
lines = 65
print(" " + lines*"-")
print(f"|{'Eficiencia de Ethernet(tcp/ip)'.center(lines)}|")
print(" " + lines*"-")
print(f"| Protocolo | Tamaño (bits) | Carga util (bits) |  Eficiencia (%) |")
print(" " + lines*"-")
print(f"| TCP/IP    |{bytes_tcp_max:14,.0f} |{carga_util_tcp:18,.0f} |{eficiencia_tcp:14,.1f} % |")
print(f"| Ethernet  |{bytes_eth_max:14,.0f} |{carga_util_eth:18,.0f} |{eficiencia_eth:14,.1f} % |")
print(" " + lines*"-")
print(f"| Total     |{bytes_eth_max:14,.0f} |{carga_util_tcp:18,.0f} |{eficiencia:14,.1f} % |")
print(" " + lines*"-")


 -----------------------------------------------------------------
|                  Eficiencia de Ethernet(tcp/ip)                 |
 -----------------------------------------------------------------
| Protocolo | Tamaño (bits) | Carga util (bits) |  Eficiencia (%) |
 -----------------------------------------------------------------
| TCP/IP    |         1,500 |             1,448 |          96.5 % |
| Ethernet  |         1,622 |             1,500 |          92.5 % |
 -----------------------------------------------------------------
| Total     |         1,622 |             1,448 |          89.3 % |
 -----------------------------------------------------------------


In [65]:
# FTP
''' La velocidad neta disponible para el usuario en una aplicación FTP, 
suele ser entre un 10% y un 15% menor a la velocidad real, 
debido al ancho de banda consumido por las cabeceras y las colas 
de los distintos protocolos usados en una transmisión libre de errores.'''
tamaño_del_archivo = 100_000_000    # Archivo de prueba.
tamaño_de_la_ventana = 4096         # Especifica el tamaño del búfer de transferencia.
colisiones = 1                      # Porcentaje de segmentos retransmitidos (1-5).
retardo_esperado = 50               # LAN:30µs, FO:10ms, CATV:20ms, ADSL:50ms, Satélite:900ms.
tiempo_ms = 2_471

# ftp
tiempo_comandos_ftp = 3*retardo_esperado
ventanas = m.ceil(tamaño_del_archivo/tamaño_de_la_ventana)
paquetes_por_ventana = m.ceil(tamaño_de_la_ventana/carga_util_tcp)
total_paquetes_tcp = (1+colisiones/100) * paquetes_por_ventana * ventanas

# Tasas de transferencia
tiempo = (tiempo_ms-tiempo_comandos_ftp)/1_000
throughput = (total_paquetes_tcp * bytes_tcp_max) / tiempo
goodput = tamaño_del_archivo / tiempo
sobrecarga = -100*(1-throughput/goodput)

# Salida estándar - Formateo de cadenas
lines = 35
print(lines*"-")
print(f"Tiempo transferencia:   {tiempo*1e3:11,.0f} ms")
print(f"Carga total:            {tamaño_del_archivo/1e6:11,.0f} Mbps")
print(f"Throughput:             {throughput/1e6:11,.0f} Mbps")
print(f"Goodput:                {goodput/1e6:11,.0f} Mbps")
print(f"Sobrecarga:             {sobrecarga:11,.1f} %")
print(lines*"-")

-----------------------------------
Tiempo transferencia:         2,321 ms
Carga total:                    100 Mbps
Throughput:                      48 Mbps
Goodput:                         43 Mbps
Sobrecarga:                    11.0 %
-----------------------------------


### 2.1. Tasa de transferencia de símbolos 
En un sistema de comunicaciones la [tasa de transferencia de símbolos][21] o Rs es el número de unidades de señal por unidad de tiempo que pueden ser transmitidos desde un emisor hacia el receptor a traves de un canal de comunicaciones.

[21]:https://es.wikipedia.org/wiki/Tasa_de_baudios

#### 2.2.1 Consideraciones
* También llamado [baudrate][221_1].
* Define el [número de símbolos][221_2] que se transmiten por unidad de tiempo.
* El cambio de estado en [señal digital][221_3] es un símbolo.
* Dependiendo de la [modulación][221_4] un símbolo puede representar muchos bits.
* Esta limitado por el [ancho de banda][221_5].
* Se mide en [Baudio][221_6] (b/s).

[221_1]:https://es.wikipedia.org/wiki/Tasa_de_baudios
[221_2]:https://es.wikipedia.org/wiki/C%C3%B3digos_en_l%C3%ADnea
[221_3]:https://es.wikipedia.org/wiki/Se%C3%B1al_digital
[221_4]:https://es.wikipedia.org/wiki/Modulaci%C3%B3n_(telecomunicaci%C3%B3n)/
[221_5]:https://es.wikipedia.org/wiki/Ancho_de_banda_(inform%C3%A1tica)/
[221_6]:https://es.wikipedia.org/wiki/Baudio


#### 2.2.2. Tasa de transferencia binaria (Rb) ✔
* También llamado [bitrate][222_1].
* Define el número de bits que se transmiten por unidad de tiempo.
* Depende de la [codificación del canal][222_2] y el ancho de banda.
* Debe ser menor a la capacidad del canal.
* Se mide en [bits por segundo][222_6] (b/s). 

[222_1]:https://es.wikipedia.org/wiki/Tasa_de_bits
[222_2]:https://es.wikipedia.org/wiki/C%C3%B3digo_de_canal
[222_6]:https://es.wikipedia.org/wiki/Bits_por_segundo

In [67]:
# Gigabit Ethernet 1000Base-T 
# Line code TCM 4D-PAM-5
carriles = 4
cambios_señal = 5       # Símbolos (niveles)
simbolos = 5
ancho_de_banda = 62.5



rendimiento_por_carril = 4
throughput = rendimiento_por_carril * carriles * ancho_de_banda
throughput

1000.0

## 3. Capacidad del canal  ✔
El [Ruido][noise] son las potencias parásitas que afectan la transmisión.

[noise]:https://es.wikipedia.org/wiki/Ruido_(comunicación)#

### 3.1. Consideraciones
* Maxima [información][22_1] que un canal puede transmitir de forma fiable.
* Fiable significa que la [probabilidad de error][22_2] es despreciable. 
* Está limitada por su [relación señal a ruido][22_4] (SNR) y el ancho de banda.
* Se calcula empleando el [teorema de Shannon-Hartley][22_5].
* Se mide en [bits por segundo][22_6] (b/s). 

[22_1]:https://es.wikipedia.org/wiki/Informaci%C3%B3n
[22_2]:https://es.wikipedia.org/wiki/Teorema_de_Shannon-Hartley

[22_4]:https://es.wikipedia.org/wiki/Relaci%C3%B3n_se%C3%B1al/ruido
[22_5]:https://es.wikipedia.org/wiki/Teorema_de_Shannon-Hartley
[22_6]:https://es.wikipedia.org/wiki/Bits_por_segundo

### 3.2. Ecuaciones de Shannon - Hartley

$$
\begin{aligned}
C = B \cdot log_{2}\left(1+SNR\right) &= B \cdot log_{2}\left(1+ \frac{P_S}{P_N}\right) \\
&= B \cdot log_{2}\left(1+ \frac{E_b \cdot R_b}{N_o \cdot B}\right) \\ 
\end{aligned}
$$

### 3.3. Representación

![Representación](../img/channel_capacity.svg "Ca")

In [6]:
Vc = 299_792_458        # Velocidad de la luz [m.s]
Kp = 6.79e-34           # Constante de Plank [J.s]

# Diccionario de frecuencias de la luz en Thz
frecuencias = {"Infrarrojo":337, "Rojo":470, "Amarillo":496, "Verde":581, "Azul": 641, "Violeta":744}

print(f"------------------------------------------------------------------------")
print(f"|  Color de la Luz | FRECUENCIA | LONGITUD ONDA |  ENERGÍA  | POTENCIA |")
print(f"------------------------------------------------------------------------")
for key in frecuencias:
    color = key
    f = frecuencias[key] * 1e12
    λ = Vc / f
    e = Kp * f
    p = 3600 * e
    print(f"|       {color:>10} | {f:10.2e} |  {λ:11.2e}  |  {e:7.2e} | {p:7.2e} |")

print(f"------------------------------------------------------------------------")

------------------------------------------------------------------------
|  Color de la Luz | FRECUENCIA | LONGITUD ONDA |  ENERGÍA  | POTENCIA |
------------------------------------------------------------------------
|       Infrarrojo |   3.37e+14 |     8.90e-07  |  2.29e-19 | 8.24e-16 |
|             Rojo |   4.70e+14 |     6.38e-07  |  3.19e-19 | 1.15e-15 |
|         Amarillo |   4.96e+14 |     6.04e-07  |  3.37e-19 | 1.21e-15 |
|            Verde |   5.81e+14 |     5.16e-07  |  3.94e-19 | 1.42e-15 |
|             Azul |   6.41e+14 |     4.68e-07  |  4.35e-19 | 1.57e-15 |
|          Violeta |   7.44e+14 |     4.03e-07  |  5.05e-19 | 1.82e-15 |
------------------------------------------------------------------------


---
## Mas Recursos
- [Ancho de banda](https://es.wikipedia.org/wiki/Ancho_de_banda) (Wikipedia)
- [Producto: retardo * ancho de banda](https://en.wikipedia.org/wiki/Bandwidth-delay_product) (Wikipedia)
- [Ethernet](https://es.wikipedia.org/wiki/Ethernet) (Wikipedia)
- [HTTP](https://es.wikipedia.org/wiki/Protocolo_de_transferencia_de_hipertexto) (Wikipedia)
